# RatU Day4SD recording info
- No timestamps were deleted after concatenating .dat files from various folders of open-ephys

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import subjects

sess = subjects.sd.ratUday4[0]
print(sess.recinfo)

filename: /data/Clustering/sessions/RatU/RatUDay4SD/RatU_Day4SD_2021-07-29_08-23-06.xml 
# channels: 192
sampling rate: 30000
lfp Srate (downsampled): 1250



# Set probe configuration
- RatU_Day4SD has two probes: both 128chan-8shanks diagnostic biochips.
- 64 channels in one of the probes (implanted in left hemisphere) had no signal from 4 shanks (probably one of the intan chips was faulty). So only 192 channels were recorded, the channels in .dat file are already order according to depth.


In [ ]:
%matplotlib inline
from neuropy.core import shank, probe, probegroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

#--- diagnostic-biochip 8 shanks -----------
for i in range(8):
    chans = channel_groups[i]
    shank = shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=15,
        y_shift_per_column=[0,-7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = probe(shanks)

#--- dignostic biochip 4 shanks (4 shanks were bad) ----------- 
shanks = []
for i in range(8,12):

    shank = shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=15,
        y_shift_per_column=[0, -7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)


probe2 = probe(shanks)
probe2.move((probe1.x_max+500,0))

prbgrp = probegroup()
prbgrp.add_probe(probe1)
prbgrp.add_probe(probe2)

prbgrp.filename = sess.fileprefix.with_suffix(".probegroup.npy")
prbgrp.save()
plot_probe(prbgrp)

## Writing to json format for spyking-circus 

In [ ]:
from neuropy.utils import probe_util

file = sess.filePrefix.with_suffix('.prb')
probe_util.write_spyking_circus(file,sess.probegroup,shanksCombine=True)


# Create experimental paradigm
- pre sleep is a little shorter
- animal was lazy on re-maze

In [ ]:
sess.paradigm.to_dataframe()

In [ ]:
import pandas as pd
from neuropy.core import Epoch

datetime_data = pd.read_csv(sess.filePrefix.with_suffix('.datetime.csv'))
durations = datetime_data.nFrames/sess.recinfo.dat_sampling_rate
epochs = pd.DataFrame(
    {
        "start": [0, 9411,12547,30565,12547,45161],
        "stop": [9410, 12546, 30564, 45160,45160,48283],
        "label": ["pre", "maze", "sd",'rs','post','re-maze'],
    }
)

paradigm = Epoch(epochs=epochs)
paradigm.filename = sess.filePrefix.with_suffix(".paradigm.npy")
paradigm.save()


# Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

## Artifacts epochs
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
from neuropy.analyses import detect_artifact_epochs
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.core import Signal
print(sess)
signal = sess.eegfile.get_signal([96])
# filt_trace = signal_process.filter_sig.highpass(signal.traces,cutoff=400)
# filtered_signal = Signal(filt_trace,signal.sampling_rate)
artifact_epochs = detect_artifact_epochs(signal, thresh=7)
artifact_epochs.save(sess.filePrefix.with_suffix(".artifact.npy"))
# artifact_epochs.save()
plotting.plot_artifact_epochs(artifact_epochs,signal)


In [ ]:
28488/60


### write artifact epochs to spyking circus format

In [ ]:
from neuropy.io import SpykingCircusIO

file = sess.filePrefix.with_suffix('.dead')
SpykingCircusIO.write_epochs(file,sess.artifact)

## Sleep scoring

In [ ]:
from neuropy.core import Signal
from neuropy.analyses import brainstates

signal = sess.eegfile.get_signal() 
brainstates = brainstates.detect_brainstates_epochs(signal=signal,probe=sess.probegroup)

In [ ]:
brainstates.filename = sess.filePrefix.with_suffix('.brainstates')
brainstates.save()

## Ripple epochs
To detect ripples one also needs probegroup.

In [ ]:
from neuropy.analyses import oscillations
signal = sess.eegfile.get_signal()
ripple_epochs =oscillations.detect_ripple_epochs(signal, sess.probegroup)
ripple_epochs.filename = sess.filePrefix.with_suffix('.ripple.npy')
ripple_epochs.save()

In [ ]:
signal = sess.eegfile.get_signal(channel_id=[1, 2, 3, 4], t_start=1, t_stop=1.2)
plotting.plot_signal_traces(signal)

# Neurons

## Importing spiketrains from Phy

In [ ]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path("/home/bapung/Documents/ClusteringHub/spykcirc/RatU/RatUDay4SD/RatU_Day4SD_2021-07-29_08-23-06-1.GUI")
chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains =phy_data.spiketrains
peak_chans = phy_data.peak_channels
waveforms = phy_data.waveforms
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

neuron_type_id = phy_data.cluster_info.q.values
neuron_type = np.ones(len(neuron_type_id), dtype="U5")
neuron_type[neuron_type_id<4] = 'pyr'
neuron_type[neuron_type_id==6] = 'mua'
neuron_type[neuron_type_id==8] = 'inter'


neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms,dtype='object'),
    shank_ids=np.array(shank_id).astype(int),
    neuron_type=neuron_type,
    metadata={'cluster_path':cluster_path}
)

neurons.filename = sess.filePrefix.with_suffix('.neurons')
neurons.save()


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from neuropy.plotting import plot_raster


plt.plot(phy_data.peak_waveforms[0])
plot_raster(neurons,color='jet',add_vert_jitter=True)

## BinnedSpiketrain and Mua objects using Neurons

In [ ]:
mua =sess.neurons.get_mua()
mua.filename = sess.filePrefix.with_suffix(".mua.npy")
mua.save()   


In [ ]:
%matplotlib widget
from neuropy import plotting
smth_mua = sess.mua.get_smoothed(sigma=0.02)
plotting.plot_mua(smth_mua)

In [ ]:
from neuropy.analyses import detect_pbe_epochs

pbe = detect_pbe_epochs(smth_mua)
pbe.filename = sess.filePrefix.with_suffix('.pbe')
pbe.save()


# Position
- concatenated .dat file did not have any deleted timepoints

## Import position from optitrack

In [ ]:
from neuropy.io import OptitrackIO
from neuropy.core import Position
from pathlib import Path

opti_folder = sess.filePrefix.parent / 'position'
opti_data = OptitrackIO(dirname=opti_folder,scale_factor=0.25)

### Align position with .dat file
- t_error for maze position alignment were estimated using theta power and speed cross correlogram.
- theta channel

In [ ]:
import pandas as pd
from datetime import datetime

# ------- maze align corection ---------
t_error = [0, 0, 0.8, 0, 0, 0.680]

# ---- startimes of concatenated .dat files
tracking_sRate = opti_data.sampling_rate
rec_datetime = pd.read_csv(sess.filePrefix.with_suffix(".datetime.csv"))
data_time = []
for i, file_time in enumerate(rec_datetime["StartTime"]):
    tbegin = datetime.strptime(file_time, "%Y-%m-%d_%H-%M-%S") + pd.Timedelta(
        t_error[i], unit="sec"
    )
    nframes = rec_datetime["nFrames"][i]
    duration = pd.Timedelta(nframes / sess.recinfo.dat_sampling_rate, unit="sec")
    tend = tbegin + duration
    trange = pd.date_range(
        start=tbegin,
        end=tend,
        periods=int(duration.total_seconds() * tracking_sRate),
        closed="left",
    )
    data_time.extend(trange)
data_time = pd.to_datetime(data_time)

x, y, z = opti_data.get_position_at_datetimes(data_time)
traces = np.vstack((z, x, y))

position = Position(traces=traces, t_start=0, sampling_rate=opti_data.sampling_rate)
position.save(sess.filePrefix.with_suffix(".position.npy"))

_, ax = plt.subplots()
ax.plot(position.x, position.y)


## Linearize position

In [ ]:
from neuropy.utils import position_util

for e in ['re-maze']:
    maze = sess.paradigm[e].flatten()
    maze_pos = sess.position.time_slice(maze[0],maze[1])
    linear_pos = position_util.linearize_position(maze_pos)
    e = e.replace('-','')
    # linear_pos.save(sess.filePrefix.with_suffix(f'.{e}.linear'))


In [ ]:
import matplotlib.pyplot as plt
_,ax = plt.subplots()
ax.plot(linear_pos.time,linear_pos.x)

In [ ]:
from shapely.geometry import LineString,Polygon

line = LineString([(0,0),(1,1)])

In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

maze = sess.paradigm['maze'].flatten()
position = sess.position.time_slice(maze[0],maze[1])
x,y = position.x,position.y

# line = Polygon(zip(x[::30],y[::30]))

xbin = np.arange(x.min(),x.max(),2)
ybin = np.arange(y.min(),y.max(),2)
occupancy = np.histogram2d(x,y,bins=[xbin,ybin])[0]
occupancy = occupancy/60
# occupancy = np.sqrt(occupancy)
# occupancy = gaussian_filter(occupancy,sigma=2)

x_max,y_max = np.where(occupancy>=0.5) 

# plt.plot(x,y,'r')
plt.pcolormesh(xbin[:-1],ybin[:-1],np.flipud(np.rot90(occupancy)),shading='gouraud',vmax=5)
plt.plot(xbin[x_max],ybin[y_max],'.')

In [ ]:
from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt
import numpy as np

maze = sess.paradigm['re-maze'].flatten()
position = sess.position.time_slice(*maze)
fig, ax = plt.subplots()
ax.plot(position.x,position.y)

coord = []

def on_click(event):
    if (event.button is MouseButton.LEFT) and (event.inaxes):
        # print(f'{event.xdata} {event.ydata}')
        # ax.plot(event.xdata,event.ydata,'r',lw=20)
        global coord

        coord.append((event.xdata, event.ydata))
        x = event.xdata
        y = event.ydata
        
        ax.plot(x,y,'o',color='r')
        fig.canvas.draw() #redraw the figure
    
    if (event.button is MouseButton.RIGHT):
        fig.disconnent()
    


        # plt.disconnect(binding_id)

fig.canvas.mpl_connect('button_press_event', on_click)



In [ ]:
from shapely.geometry import LineString
line = LineString(coord)

In [ ]:
line.length

In [ ]:
distance_delta = 2
distances = np.arange(0, line.length, distance_delta)
# or alternatively without NumPy:
# points_count = int(line.length // distance_delta) + 1
# distances = (distance_delta * i for i in range(points_count))
points = [line.interpolate(distance) for distance in distances] + [line.boundary[1]]
# multipoint = unary_union(points)  # or new_line = LineString(points)
new_line = LineString(points)

In [ ]:
_,ax = plt.subplots()
plt.plot(*line.xy,'r.')
plt.plot(*new_line.xy,'k.')

In [ ]:
from shapely.geometry import Point

lin_pos = []
for x,y in zip(position.x,position.y):
    lin_pos.append(line.project(Point(x,y)))


In [ ]:
from neuropy.core import Position
from scipy.ndimage import gaussian_filter1d

lin_pos = np.array(lin_pos).reshape(1,-1)
lin_pos = gaussian_filter1d(lin_pos,sigma=6)
linear_position = Position(traces=lin_pos,t_start=maze[0],sampling_rate=position.sampling_rate,metadata={'method':'shapely linearization'}) 
linear_position.save(sess.filePrefix.with_suffix('.remaze.linear'))

In [ ]:
_,ax = plt.subplots()
ax.plot(linear_position.time,linear_position.x)
ax.plot(linear_position.time,linear_position.speed)

In [ ]:
lin_pos.shape

In [ ]:
plt.plot(position.time,lin_pos)
plt.plot(position.time,position.x)
# plt.plot(sess.lin_maze.time,sess.lin_maze.x)

In [ ]:
from hfuncs import linearize_using_shapely

maze = sess.paradigm['maze'].flatten()
linearize_using_shapely(sess.position.time_slice(*maze))